 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="#API-Overview" data-toc-modified-id="API-Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>API Overview</a></span><ul class="toc-item"><li><span><a href="#Some-Context-[Optional]" data-toc-modified-id="Some-Context-[Optional]-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Some Context [Optional]</a></span></li><li><span><a href="#Create-a-ClipperConnection" data-toc-modified-id="Create-a-ClipperConnection-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Create a ClipperConnection</a></span></li><li><span><a href="#Start-Clipper" data-toc-modified-id="Start-Clipper-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Start Clipper</a></span></li><li><span><a href="#Deploy-a-model" data-toc-modified-id="Deploy-a-model-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Deploy a model</a></span><ul class="toc-item"><li><span><a href="#Create-the-model" data-toc-modified-id="Create-the-model-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Create the model</a></span><ul class="toc-item"><li><span><a href="#Solution" data-toc-modified-id="Solution-1.4.1.1"><span class="toc-item-num">1.4.1.1&nbsp;&nbsp;</span>Solution</a></span></li></ul></li><li><span><a href="#Deploy-to-Clipper" data-toc-modified-id="Deploy-to-Clipper-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Deploy to Clipper</a></span></li><li><span><a href="#A-Note-About-Types-[Optional]" data-toc-modified-id="A-Note-About-Types-[Optional]-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>A Note About Types [Optional]</a></span></li></ul></li><li><span><a href="#Register-an-application" data-toc-modified-id="Register-an-application-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Register an application</a></span></li><li><span><a href="#Inspecting-Clipper" data-toc-modified-id="Inspecting-Clipper-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Inspecting Clipper</a></span></li><li><span><a href="#Updating-the-Model" data-toc-modified-id="Updating-the-Model-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Updating the Model</a></span></li><li><span><a href="#Adding-Model-Replicas" data-toc-modified-id="Adding-Model-Replicas-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Adding Model Replicas</a></span></li></ul></li><li><span><a href="#Example-Application---Birds-vs-Airplanes" data-toc-modified-id="Example-Application---Birds-vs-Airplanes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Example Application - Birds vs Airplanes</a></span><ul class="toc-item"><li><span><a href="#Load-Cifar" data-toc-modified-id="Load-Cifar-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load Cifar</a></span></li><li><span><a href="#Create-an-application" data-toc-modified-id="Create-an-application-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Create an application</a></span></li><li><span><a href="#Start-serving" data-toc-modified-id="Start-serving-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Start serving</a></span></li><li><span><a href="#Train-Logistic-Regression-Model" data-toc-modified-id="Train-Logistic-Regression-Model-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Train Logistic Regression Model</a></span></li><li><span><a href="#Deploy-Logistic-Regression-Model" data-toc-modified-id="Deploy-Logistic-Regression-Model-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Deploy Logistic Regression Model</a></span></li><li><span><a href="#Load-TensorFlow-Model" data-toc-modified-id="Load-TensorFlow-Model-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Load TensorFlow Model</a></span></li><li><span><a href="#Deploy-TensorFlow-Model" data-toc-modified-id="Deploy-TensorFlow-Model-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Deploy TensorFlow Model</a></span></li></ul></li><li><span><a href="#Restarting-Clipper" data-toc-modified-id="Restarting-Clipper-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Restarting Clipper</a></span></li></ul></div>

In [ ]:
from __future__ import absolute_import, division, print_function
import logging
import sys
import os
import time
import subprocess32 as subprocess
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# API Overview

In the first part of this exercise, you will explore how to create and interact with a Clipper cluster. The primary way of managing Clipper is with the Clipper Admin Python tool. This tutorial will walk you through all the things you can do with the Clipper Admin tool as well as explain what happens within Clipper when you issue each command. You can find the complete API documentation for the Clipper Admin tool on our website: <http://docs.clipper.ai>.

**Goal:** Be familiar with how to create and manage a Clipper cluster, and understand what happens when you issue Clipper admin commands.

## Some Context [Optional]

The Clipper Admin tool is distributed through Pip. You can install it with `pip install clipper_admin`, but it has already been installed in this notebook for you.

Clipper is built on top of Docker containers. A running Clipper cluster consists of a collection of Docker containers communicating with each other over the network. As you issue commands against Clipper, you are communicating with these containers as well as creating new ones or destroying existing ones. As you explore the Clipper API throughout this exercise, we will illustrate how each command effects the cluster state.

The main API for interacting with Clipper is exposed via a [`ClipperConnection`](http://docs.clipper.ai/en/develop/#clipper-connection) object. This is your handle to a Clipper cluster (this collection of Docker containers). It can be used to start, stop, inspect, and modify the cluster.

In order to create a `ClipperConnection` object, you must provide it with a [`ContainerManager`](http://docs.clipper.ai/en/develop/#container-managers). While Docker is becoming an increasingly standard mechanism for deploying applications, there are many different tools for managing a Docker cluster. These tools broadly fall into the category of *Container Orchestration frameworks*. Some popular examples are [Kubernetes](https://kubernetes.io/), [Docker Swarm](https://docs.docker.com/engine/swarm/), and [DC/OS](https://dcos.io/). One of the reasons we run Clipper in Docker containers is to make the system as general as possible and support many different deployment scenarios. Within the Clipper Admin, we abstract away all of the Docker container-specific commands behind the `ContainerManager` interface. The `ClipperConnection` object makes Clipper-specific decisions about how to issue commands, and then makes any changes to the Docker configuration (for example, to launch a container for a newly deployed model) through the `ContainerManager`. To support different container orchestration frameworks that manage Docker containers in different ways, we create different implementations of the `ContainerManager` interface.

Clipper currently provides two `ContainerManager` implementations: the `DockerContainerManager` and the `KubernetesContainerManager`. In this exercise, you will be using the `DockerContainerManager`, which runs Clipper directly on your local Docker instance. This `ContainerManager` is particularly useful for trying out Clipper without needing to set up an enterprise-grade container orchestration framework. The `DockerContainerManager` is not recommended for production use cases.

## Create a ClipperConnection

First, create a new [`ClipperConnection`](http://docs.clipper.ai/en/develop/#clipper-connection) object with the type of `ContainerManager` you want to use. Simply creating a new connection object does not connect to Clipper. This is a good thing. You haven't started Clipper yet so you would have nothing to connect to!

>*The command to get the Docker IP address is a bit of hack to deal with the fact that you are running this exercise inside its own Docker container. Without going into the details of how Docker handles container networking, what this command does is figure out how to send commands to "localhost" on the *host machine* from inside a Docker container. If you're curious, [this StackOverflow answer](https://stackoverflow.com/a/24326540/814642) provides a nice overview of the problem and the solution.*

> *Generally, you will not have to deal with this and can leave the `docker_ip_address` argument as the default of `127.0.0.1` (localhost).*


In [ ]:
from clipper_admin import ClipperConnection, DockerContainerManager
docker_ip = subprocess.check_output("./get_docker_ip.sh").strip()
clipper_conn = ClipperConnection(DockerContainerManager(docker_ip_address=docker_ip))

## Start Clipper

This command will start three Docker containers:

1. The query frontend: This container listens for incoming prediction requests and schedules and routes them to the deployed models.
2. The management frontend: This container manages and updates Clipper's internal configuration state.
3. A Redis instance: Redis is used to persistently store Clipper's internal configuration state. You started Redis on port 6380 instead of the default port to avoid collisions with any Redis instances that are already running.

These containers are networked together as illustrated below.

<img src="img/docker_config_imgs/start_clipper.png" style="width: 600px;"/>

> *Because Docker must download the Docker images from the internet (if they are not already cached) before it can start the containers, the first time you run this command can take a long time to complete (up to a couple minutes) while the image is downloaded. Thanks for your patience.*

If you try to start more than one Clipper cluster at once on the same host, the second execution of the command will fail because, by default, the second cluster will try to bind to the same ports as the first one. If you run into problems with the exercise and want to start over, see [Section 3](#Restarting-Clipper) for instructions on how to reset Clipper.

In [ ]:
clipper_conn.start_clipper()
clipper_addr = clipper_conn.get_query_addr()

If you now list the running Docker containers, filtering out any containers not started by Clipper, you should see the three Clipper containers listed (look at the "IMAGE" field of the output).

In [ ]:
!docker ps --filter label=ai.clipper.container.label

## Deploy a model


At its most basic, a trained model is just a function that takes some input and produces some output. As a result, one way to think about Clipper is as a function server. While these functions are often complex models, Clipper is not restricted to serving machine learning models.

To start with, you will deploy a very simple function to Clipper. You'll start with the classic "Big Data" version of Hello World: *Word Count.*

### Create the model

Before you can deploy a model to Clipper, you have to write it.

Define a function that takes in a document (a string of text) and returns the number of words in the text as an integer.

```py
def count_words()
```

In [ ]:
def count_words(text):
    raise NotImplementedError

In [ ]:
test_one = "Hello world. Foo bar baz."
assert count_words(test_one) == 5
test_two = "I'm learning so much at the inaugural RISE Camp."
assert count_words(test_two) == 9

To improve performance during inference, many machine learning models exploit opportunities for data parallelism in the inference process. Because of this, Clipper tries to provide multiple inputs at once to a deployed model. Therefore, models deployed to Clipper must have a function interface that takes a list of inputs as an argument and returns a list of predictions as strings. Returning predictions as strings provides a lot of flexibility over what your models can return. Commonly, models in Clipper will return either a single number (such as a label or score) or JSON containing a richer representation of the model output (for example, by including confidence estimates of predicted labels).

To adapt your word count example to match the required interface, define a second function that takes a list of documents as an argument and returns the word count of each document as a string.

In [ ]:
# The function you deploy to Clipper must take a list of inputs, so we
# wrap our core word-count logic in a loop.
def count_words_in_docs(docs):
    counts = []
    for d in docs:
        count = str(count_words(d))
        counts.append(count)
    return counts

In [ ]:
assert count_words_in_docs([test_one, test_two]) == ["5", "9"]

#### Solution

```py
def count_words(text):
    words = text.split()
    return len(words)
```

### Deploy to Clipper

One of the goals of Clipper is to make it simple to deploy and maintain machine-learning models in production. The prediction interface that models must implement is very simple, consisting of a single function. And the use of Docker makes it easy to include all of a model's dependencies in a self-contained environment. However, deploying a new type of model still entails writing and debugging a new model container and creating a Docker image.

To make the model deployment process even simpler, Clipper provides a library of *model deployers* for common types of models. If your model can be deployed with one of these deployers, you no longer need to write a model container, create a Docker image, or even figure out how to save a model. Instead, you provide your trained model directly to the model deployer function within your Python process. The model deployer takes care of saving the model and building a Docker image that is compatible with your model type.

Currently, Clipper provides two model deployers, one to deploy arbitrary Python functions (within some constraints) and the other to deploy PySpark models along with pre- and post-processing logic. In this exercise, you will be using the Python deployer.

[Read more about model deployers in the Clipper documentation.](http://docs.clipper.ai/en/develop/#model-deployers)

In [ ]:
from clipper_admin.deployers import python as python_deployer
python_deployer.deploy_python_closure(
    clipper_conn,
    name="wordcount",  # The name of the model in Clipper
    version=1,  # A unique identifier to assign to this model.
    input_type="string",  # The type of data the model function expects as input
    func=count_words_in_docs # The model function to deploy
)

Clipper deploys each model in its own Docker container. After deploying the model, Clipper uses the `DockerContainerManager` to start a container for this model and create an RPC connection with the Clipper query frontend, as illustrated below (the changes to the cluster are highlighted in red).

> *Once again, Clipper must download a Docker container from the internet the first time this command is run.*

<img src="img/docker_config_imgs/deploy_model.png" style="width: 600px;"/>


If you list the Clipper containers again, you can see the container running your word count model.

In [ ]:
!docker ps --filter label=ai.clipper.container.label

### A Note About Types [Optional]

When you deploy models and register applications, you must specify the input type that the model or application expects. The type that you specify has implications for how Clipper manages input serialization and deserialization. From the user's perspective, the input type affects the behavior of Clipper in two places. In the "input" field of the request JSON body, applications will reject requests where the value of that field is the wrong type. And the deployed model function will be called with a list of inputs of the specified type.

The input type can be one of the following types:
+ *"ints"*: The value of the "input" field in a request must be a JSON list of ints. The model function will be called with a list of numpy arrays of type `numpy.int`.
+ *"floats"*: The value of the "input" field in a request must be a JSON list of doubles. The model function will be called with a list of numpy arrays of type `numpy.float32`.
+ *"doubles"*: The value of the "input" field in a request must be a JSON list of doubles. The model function will be called with a list of numpy arrays of type `numpy.float64`.
+ *"bytes"*: The value of the "input" field in a request must be a Base64 encoded string. The model function will be called with a list of numpy arrays of type `numpy.int8`.
+ *"strings"*: The value of the "input" field in a request must be a string. The model function will be called with a list of strings.

## Register an application

You've now deployed a model to Clipper, but you don't have any way to query it yet. Instead of automatically creating a REST endpoint when you deploy a model, Clipper introduces a layer of indirection: the application. Clients query a specific application in Clipper, and the application routes the query to the correct model. This allows multiple applications to route queries to the same model, and in the future will allow a single application to route queries to multiple models. A single Clipper cluster can have many applications registered and many models deployed at once.

When you register an application you configure certain elements of the application's behavior. These include:
+ The name to give the REST endpoint.
+ The input type that the application expects (Clipper will ensure applications only route requests to models with matching input types).
+ The latency service level objective (SLO) specified in microseconds. Clipper will manage how it schedules and routes queries for an application based on the specified service level objective. For example, Clipper will set the amount of time it allows requests to spend queued before being sent to the model based on the service level objective for the application requesting the prediction. In addition, Clipper will respond to requests by the end of the specified SLO, even if it has not received a prediction back from the model.
+ The default output: Clipper will respond with the default output to requests if a real prediction isn't available by the end of the service level objective.


When you register an application with Clipper, it creates a REST endpoint for that application:

```
URL: /<app_name>/predict
Method: POST
Data Params: {"input": <input>}
```

<img src="img/docker_config_imgs/register_app.png" style="width: 600px;"/>

Register an application to query your word count function:

In [ ]:
clipper_conn.register_application(
    name="wordcount-app",
    input_type="strings",
    default_output="-1",
    slo_micros=100000)

Try querying the newly created application with [curl](https://curl.haxx.se/).

In [ ]:
%%bash -s "$clipper_addr"
curl -s -X POST --header "Content-Type:application/json" \
    -d '{"input": "The sky above the port was the color of television, tuned to a dead channel."}' \
    $1/wordcount-app/predict

You should see that your application returned the default output of "-1". This is because even though you have deployed a model and registered an application, you have not told Clipper to route requests from the "wordcount-app" application to the "wordcount" model.

You do this by *linking* the model to the application.

<img src="img/docker_config_imgs/link_model.png" style="width: 600px;"/>


In [ ]:
clipper_conn.link_model_to_app(app_name="wordcount-app", model_name="wordcount")

When you query the "wordcount-app" endpoint again, Clipper should return the correct word count. Try it with your own phrase.

In [ ]:
%%bash -s "$clipper_addr"
curl -s -X POST --header "Content-Type:application/json" \
    -d '{"input": "TRY YOUR OWN PHRASE"}' \
    $1/wordcount-app/predict

## Inspecting Clipper

The `ClipperConnection` object has several methods to inspect various aspects of the Clipper cluster.

You can list all of the applications.

In [ ]:
clipper_conn.get_all_apps(verbose=True)

Or all of the models.

In [ ]:
clipper_conn.get_all_models(verbose=True)

Clipper also tracks several performance metrics that you can inspect at any time.

In [ ]:
clipper_conn.inspect_instance()

You can also fetch the raw container logs from all of the Clipper docker containers. The command will print the paths to the log files for further examination. You can figure out which logs belong to which container based on the unique Docker container ID in the log filename.

In [ ]:
clipper_conn.get_clipper_logs()

## Updating the Model

Machine learning models are rarely static. Instead, data science tends to be an iterative process, with new and improved models being developed over time. Clipper supports this workflow by letting you deploy new versions of models. If you look back to where you linked your wordcount model to the application, you'll see that there is no mention of versioning in that method call. Instead, when a new version of a model is deployed, Clipper will automatically start routing requests to the new version.

Create a new version of the "wordcount" model that counts the number of *unique* words in a document.

In [ ]:
import json
import string
def unique_word_count(text):
    # Convert to lower case
    text = text.lower()
    text = text.translate(None, string.punctuation)
    words = text.split()
    counts = {}
    for w in words:
        if w in counts:
            counts[w] += 1
        else:
            counts[w] = 1
    return len(counts.keys())
    
def count_unique_words_multiple_docs(docs):
    counts = []
    for d in docs:
        count = unique_word_count(d)
        counts.append(str(count))
    return counts

In [ ]:
word_freq_example = "Hello world. This is an example sentence. This is another sentence. There are some repeated words in this document."
count_unique_words_multiple_docs([word_freq_example])

Deploy this new version of the function as version "2". For this application, you are using a numeric versioning scheme. But Clipper just treats versions as unique string identifiers, so you could use other versioning schemes (such as Git hashes or semantic versioning). Versions don't even have to be ordered, Clipper just tracks the currently active version.

<img src="img/docker_config_imgs/update_model.png" style="width: 600px;"/>

In [ ]:
python_deployer.deploy_python_closure(
    clipper_conn,
    name="wordcount",
    version="2",
    input_type="string",  # The input type must match across all model versions
    func=count_unique_words_multiple_docs # The new function
)

In [ ]:
%%bash -s "$clipper_addr"
curl -s -X POST --header "Content-Type:application/json" \
    -d '{"input": "The sky above the port was the color of television, tuned to a dead channel."}' \
    $1/wordcount-app/predict  

Sometimes the "new and improved" model is not actually improved. If you deploy a model that isn't working well, you can roll back to any previous version. This just changes which version of the model application's route requests to.

<img src="img/docker_config_imgs/rollback_version.png" style="width: 600px;"/>

In [ ]:
clipper_conn.set_model_version(name="wordcount", version="1")

In [ ]:
%%bash -s "$clipper_addr"
curl -s -X POST --header "Content-Type:application/json" \
    -d '{"input": "The sky above the port was the color of television, tuned to a dead channel."}' \
    $1/wordcount-app/predict

## Adding Model Replicas

Many machine learning models are computationally expensive and a single instance of the model may not meet the throughput demands of a serving workload. To increase prediction throughput, you can add additional replicas of a model. This creates additional Docker containers running the same model. Clipper will act as a load-balancer and distribute incoming requests across the set of available model replicas to provide higher throughput.

Set the number of replicas for the currently active version ("1") of the "wordcount" model to 4.

<img src="img/docker_config_imgs/add_replicas.png" style="width: 600px;"/>

In [ ]:
clipper_conn.set_num_replicas("wordcount", num_replicas=4)

If you list the Clipper Docker containers, you should now see four containers based on the image "wordcount:1".

In [ ]:
!docker ps --filter label=ai.clipper.container.label

If you want to reduce the number of replicas of a model to free up hardware resource, you can use the same command.

Set the number of replicas for "wordcount" back to 1.

<img src="img/docker_config_imgs/set_replicas.png" style="width: 600px;"/>

In [ ]:
clipper_conn.set_num_replicas("wordcount", num_replicas=1)

In [ ]:
!docker ps --filter label=ai.clipper.container.label

# Example Application - Birds vs Airplanes

In the second part of this exercise, you will build a real machine learning application that uses computer vision models to classify images, including training your own models.

You will create an application that labels images from the CIFAR-10 dataset as containing either birds or planes.

These images have already been downloaded and are available locally at `~/cifar`.

## Load Cifar

The first step in building any application, using machine-learning or otherwise, is to understand the application requirements. Load the dataset into the notebook so you can examine it and better understand the dataset you will be working with. The `cifar_utils` module provides some utilities for working with CIFAR data – we will make use of one of them here to load the data.

In [ ]:
import cifar_utils

cifar_loc = os.path.expanduser("cifar/")
test_x, test_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=True))
train_x, train_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_train.data", norm=True))
raw_x, raw_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=False))

Take a look at the data you've loaded. The size and blurriness of these images should give you a better understanding of the difficulty of the task you will ask of your machine learning models! If you'd like to see more images, increase the number of rows of images displayed -- the last argument to the function -- to a number greater than 2.

In [ ]:
%matplotlib inline
cifar_utils.show_example_images(raw_x, raw_y, 2)

## Create an application

For this tutorial, create an application named "cifar-binary-classifier". Note that Clipper allows you to create the application before deploying any models.

In [ ]:
app_name = "cifar-binary-classifier"
# If the model (which we will later link to our application) doesn't
# return a prediction in time, predict label 0 (bird) by default
default_output = "0"

clipper_conn.register_application(
    name=app_name,
    input_type="doubles",
    default_output=default_output,
    slo_micros=100000)

When you list the applications registered with Clipper, you should see the newly registered "cifar-binary-classifier" application show up!

In [ ]:
clipper_conn.get_all_apps()

## Start serving

Now that you have registered an application, you can start querying the application for predictions. 

You will start querying Clipper with a simple Python frontend app that computes the average accuracy of the responses after every 100 requests and updates a plot of the results with every iteration.

**[Go to the query_cifar notebook to start the app.](query_cifar.ipynb) Make sure to leave the query_cifar notebook open and the cell running as you complete the rest of this exercise.**

## Train Logistic Regression Model

When tackling a new problem with machine learning, it's always good to start with simple models and only add complexity when needed. Start by training a logistic regression binary classifier using [Scikit-Learn](http://scikit-learn.org/). This model gets about 68% accuracy on the offline evaluation dataset if you use 10,000 training examples.

In [ ]:
from sklearn import linear_model as lm 
def train_sklearn_model(m, train_x, train_y):
    m.fit(train_x, train_y)
    return m
lr_model = train_sklearn_model(lm.LogisticRegression(), train_x, train_y)
print("Logistic Regression test score: %f" % lr_model.score(test_x, test_y))

## Deploy Logistic Regression Model

While 68% accuracy on a CIFAR binary classification task is significantly below state of the art, it's already much better than the 50% accuracy your application yields right now by guessing randomly.

To deploy your Scikit-Learn logistic regression model, you can use the Python model deployer module you've been using throughout this tutorial. In particular, because the Scikit-Learn model can be pickled (the Python object serialization framework), you can wrap it in a function closure and deploy that function directly as a model to Clipper without needing to manually save the model or write a Docker
container.

First, you will write and test the prediction function to deploy. This function must take a list of inputs (a list of CIFAR images in this example) as the only function argument and return a list of predictions as strings.

In [ ]:
def sklearn_predict(images):
    preds = lr_model.predict(images)
    return [str(p) for p in preds]

print("Predicted labels: {}".format(sklearn_predict(test_x[0:3])))
print("Correct labels: {}".format(test_y[0:3]))

Now that you have defined and tested your prediction function, deploy it to Clipper as a model named "cifar-model". This time, specify the "input_type" field as "doubles." Don't forget to link the model to the application when after you've deployed it!

In [ ]:
model_name = "cifar-model"

python_deployer.deploy_python_closure(
    clipper_conn,
    name="cifar-model",
    version="1",
    input_type="doubles",
    func=sklearn_predict
)

clipper_conn.link_model_to_app(app_name="cifar-binary-classifier", model_name="cifar-model")

Now that you've deployed and linked your model to your app, go ahead and check back on your running frontend application in the [query_cifar notebook](query_cifar.ipynb). You should see the accuracy rise from around 50% to the accuracy of your SKLearn model (68%), without having to stop or modify your application at all!

## Load TensorFlow Model

To improve the accuracy of your application further, you will now deploy a TensorFlow convolutional neural network. This model takes a few hours to train on a GPU, so you will download the trained model parameters rather than training it from scratch. This model gets about 88% accuracy on the test dataset.

We have provided a pre-trained TensorFlow model stored at `tf_cifar_model/cifar10_model_full`.

In [ ]:
import os
import tensorflow as tf
import numpy as np
tf_cifar_model_path = os.path.abspath("tf_cifar_model/cifar10_model_full")
# Load the saved model
tf_session = tf.Session('', tf.Graph())
with tf_session.graph.as_default():
    saver = tf.train.import_meta_graph("%s.meta" % tf_cifar_model_path)
    saver.restore(tf_session, tf_cifar_model_path)

# Score it on the evaluation dataset
def tensorflow_score(session, test_x, test_y):
    """
    NOTE: This predict method expects pre-whitened (normalized) images
    """
    logits = session.run('softmax_logits:0',
                           feed_dict={'x:0': test_x})
    relevant_activations = logits[:, [cifar_utils.negative_class, cifar_utils.positive_class]]
    preds = np.argmax(relevant_activations, axis=1)
    return float(np.sum(preds == test_y)) / float(len(test_y))
print("TensorFlow ConvNet test score: %f" % tensorflow_score(tf_session, test_x, test_y))

## Deploy TensorFlow Model

Unlike the Scikit-Learn model, TensorFlow models cannot be pickled. Instead, they must be saved using TensorFlow's native serialization API. Because of this, you cannot use the generic Python model deployer to deploy the model to Clipper. Instead, you must save the model yourself and specify a Docker container that knows how to load and run a TensorFlow model. We have provided a Docker image for you, `clipper/tf_cifar_container:develop`, that can run the CIFAR TensorFlow model. The Docker container will load and reconstruct the model from the serialized model checkpoint when the container is started.

After completing this step and deploying the new model, Clipper will send queries to the newly-deployed TensorFlow model instead of the logistic regression Scikit-Learn model, improving the application's accuracy.

> *Once again, Clipper must download this Docker image from the internet, so this may take a minute. Thanks for your patience.*

In [ ]:
clipper_conn.build_and_deploy_model(
    name=model_name,
    version="2",
    input_type="doubles",
    model_data_path=os.path.abspath("tf_cifar_model"),
    base_image="clipper/tf_cifar_container:develop",
    num_replicas=1
)

Check out the frontend again and see if the accuracy has improved.

# Restarting Clipper

If you run into issues and want to completely stop Clipper, you can do this by calling [`ClipperConnection.stop_all()`](http://docs.clipper.ai/en/latest/#clipper_admin.ClipperConnection.stop_all).

In [ ]:
clipper_conn.stop_all()

When you list all the Docker containers a final time, you should see that all of the Clipper containers have been stopped.

In [ ]:
!docker ps --filter label=ai.clipper.container.label

You can now call `clipper_conn.start_clipper()` again without running into errors.